In [11]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import requests as req
from citipy import citipy
from datetime import datetime
from config import weather_api_key

In [ ]:
def build_url(city):
    base_url = "http://api.openweathermap.org/data/2.5/weather?"
    return base_url + "units=Imperial" + "&APPID=" + weather_api_key + "&q=" + city

In [12]:
url = "http://api.openweathermap.org/data/2.5/weather?" + "units=Imperial" + "&APPID=" + weather_api_key

In [13]:
cities = ["Pittsburgh", "Austin", "New  York", "Los Angeles", "Seattle"]
for city in cities:
    try:
        city_url = url + "&q=" + city
        weather = req.get(city_url).json()
        temp = weather['main']['temp']
    except KeyError:
        print("KeyError received for " + city)
        continue
    print("It is currently " + str(temp) + " degrees in " + city)

        

It is currently 63.66 degrees in Pittsburgh
It is currently 73.53 degrees in Austin
KeyError received for New  York
It is currently 64.08 degrees in Los Angeles
It is currently 54.7 degrees in Seattle


In [14]:
# Create an empty list to hold the weather data.
city_data = []
# Print the beginning of the logging.
print("Beginning Data Retrieval     ")
print("-----------------------------")

# Create counters.
record_count = 1
set_count = 1

# Loop through all the cities in the list.
for i, city in enumerate(cities):

    # Group cities in sets of 50 for logging purposes.
    if (i % 50 == 0 and i >= 50):
        set_count += 1
        record_count = 1
    # Create endpoint URL with each city.
    city_url = url + "&q=" + city.replace(" ","+")

    # Log the URL, record, and set numbers and the city.
    print(f"Processing Record {record_count} of Set {set_count} | {city}")
    # Add 1 to the record count.
    record_count += 1
    
    # Run an API request for each of the cities.
    try:
        # Parse the JSON and retrieve data.
        city_weather = requests.get(city_url).json()
        # Parse out the needed data.
        city_lat = city_weather["coord"]["lat"]
        city_lng = city_weather["coord"]["lon"]
        city_max_temp = city_weather["main"]["temp_max"]
        city_humidity = city_weather["main"]["humidity"]
        city_clouds = city_weather["clouds"]["all"]
        city_wind = city_weather["wind"]["speed"]
        city_country = city_weather["sys"]["country"]
        # Convert the date to ISO standard.
        city_date = datetime.utcfromtimestamp(city_weather["dt"]).strftime('%Y-%m-%d %H:%M:%S')
        # Append the city information into city_data list.
        city_data.append({"City": city.title(),
                          "Lat": city_lat,
                          "Lng": city_lng,
                          "Max Temp": city_max_temp,
                          "Humidity": city_humidity,
                          "Cloudiness": city_clouds,
                          "Wind Speed": city_wind,
                          "Country": city_country,
                          "Date": city_date})

    # If an error is experienced, skip the city.
    except:
        print("City not found. Skipping...")
        pass

# Indicate that Data Loading is complete.
print("-----------------------------")
print("Data Retrieval Complete      ")
print("-----------------------------")

Beginning Data Retrieval     
-----------------------------
Processing Record 1 of Set 1 | Pittsburgh
Processing Record 2 of Set 1 | Austin
Processing Record 3 of Set 1 | New  York
City not found. Skipping...
Processing Record 4 of Set 1 | Los Angeles
Processing Record 5 of Set 1 | Seattle
-----------------------------
Data Retrieval Complete      
-----------------------------


In [16]:
city_data_df = pd.DataFrame(city_data)
city_data_df.head(10)

,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,Pittsburgh,40.4406,-79.9959,66.2,32,90,4.61,US,2021-04-06 00:45:05
1,Austin,30.2672,-97.7431,75.0,60,75,16.11,US,2021-04-06 00:42:05
2,Los Angeles,34.0522,-118.2437,69.8,55,1,7.67,US,2021-04-06 00:50:00
3,Seattle,47.6062,-122.3321,57.0,28,1,4.61,US,2021-04-06 00:42:01


In [18]:
new_column_order = ["City", "Country", "Date", "Lat", "Lng", "Max Temp", "Humidity", "Cloudiness", "Wind Speed"]
city_data_df = city_data_df[new_column_order]
city_data_df.head()

,City,Country,Date,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed
0,Pittsburgh,US,2021-04-06 00:45:05,40.4406,-79.9959,66.2,32,90,4.61
1,Austin,US,2021-04-06 00:42:05,30.2672,-97.7431,75.0,60,75,16.11
2,Los Angeles,US,2021-04-06 00:50:00,34.0522,-118.2437,69.8,55,1,7.67
3,Seattle,US,2021-04-06 00:42:01,47.6062,-122.3321,57.0,28,1,4.61


In [19]:
# Create the output file (CSV).
output_data_file = "weather_data/cities.csv"
# Export the City_Data into a CSV.
city_data_df.to_csv(output_data_file, index_label="City_ID")

In [3]:
lats = np.random.uniform(low=-90.000, high=90.000, size=1500)
lngs = np.random.uniform(low=-180.000, high=180.000, size=1500)
lat_lngs = zip(lats, lngs)
lat_lngs

In [4]:
coordinates = list(lat_lngs)

In [5]:
city = citipy.nearest_city(22.99, 120.21)
print(city)
print(city.city_name)     # Tainan, my home town
print(city.country_code)


tainan
tw
